# Changes:
1. Based on V37
2. Refactored data augmentor. Re-implement the rotate function to preserve positive mask area.
3. Changed optimizer to adam

## Install required packages if running on google colab

In [0]:
try:
    import torch
except:
    !pip install torch torchvision
    !pip install imageio
    !pip install Augmentor
    !git clone https://github.com/allen-q/salt_oil.git
    !git clone https://github.com/allen-q/salt_net.git

In [4]:
cd salt_oil

/content/salt_oil


In [0]:
!git checkout 7b7f45978f493ae15f06202eb17c220c2c6d850d .

## Import required libs

In [0]:
import zipfile
import torch
import torchvision
import torch.nn as nn
import torch.nn.functional as F
from torch.utils import data
from torch.utils.data import Dataset, DataLoader
from torchvision import transforms, utils
from skimage import io, transform
from sklearn.model_selection import train_test_split
import numpy as np
import pandas as pd
import matplotlib.pyplot as ply
import os
import imageio
from PIL import Image
import glob
import matplotlib.pyplot as plt
import time
import math
import datetime as dt
import pytz
import pickle
from salt_func_lib import *
from torchvision import transforms, utils
from skimage import io, transform
import datetime as dt
import sys
from optparse import OptionParser
import torch.backends.cudnn as cudnn
from torch import optim
from torch.optim import lr_scheduler
import copy
from io import BytesIO
import random
import PIL
import cv2 as cv
% matplotlib inline
% load_ext autoreload
% autoreload 2

## Load Unet Modules

In [0]:
from pytorch_unet.eval import eval_net
from pytorch_unet.unet import UNet
from pytorch_unet.unet.unet_parts import *
from pytorch_unet.unet.resnet import *
from pytorch_unet.utils import get_ids, split_ids, split_train_val, get_imgs_and_masks, batch

In [0]:
## Setup data type based on whether GPU is enabled

In [8]:
if torch.cuda.is_available():
    dtype = torch.cuda.FloatTensor ## UNCOMMENT THIS LINE IF YOU'RE ON A GPU!
else:    
    dtype = torch.FloatTensor
print(f'Data Type set to: {dtype}')

Data Type set to: <class 'torch.cuda.FloatTensor'>


In [0]:
def setup_train(config_list):
    for conf in config_list:
        log.info(conf)
    for conf in config_list:
        exec(conf)

## Create Logger

In [0]:
run_name = 'salt_model_v37.42_high_lovasz_loss_cv_ens'
model_file_suffix = f'../salt_net/{run_name}_{get_current_time_as_fname()}.ckp'
fold = None

In [0]:
global log
log = get_logger(run_name)

## Prepare Data

### Load train and test data from npy files or from raw images if npy files not exist.

In [12]:
np_train_all, np_train_all_mask, X_test, misc_data = load_all_data()

Try loading data from npy and pickle files...
Data loaded.


In [0]:
df_train_mask = pd.DataFrame((np_train_all_mask/255).sum((1,2,3)), columns=['mask_pix'])
df_train_mask.mask_pix = df_train_mask.mask_pix.round(-2)

X_train_ids, X_val_ids = (
    train_test_split(df_train_mask.index.tolist(), 
                     test_size=0.20,
                     stratify = df_train_mask.mask_pix,
                     random_state=0)
)

X_train = np_train_all[X_train_ids]
X_val = np_train_all[X_val_ids]
y_train = np_train_all_mask[X_train_ids]
y_val = np_train_all_mask[X_val_ids]
depth_train = (
    misc_data['df_train_all_depth']
    .reindex(np.array(misc_data['np_train_all_ids'])[X_train_ids])
)
depth_val = (
    misc_data['df_train_all_depth']
    .reindex(np.array(misc_data['np_train_all_ids'])[X_val_ids])
)
depth_test = (
    misc_data['df_train_all_depth']
    .reindex(np.array(misc_data['np_test_ids']))
)
#X_train_mean_img = X_train.mean(0).astype(np.float32)
#X_train_mean_img = X_train.mean((0,1,2)).astype(np.float32)
X_train_mean_img = np.clip(np_train_all/255, 0, 1).mean((0,1,2)).astype(np.float32)
#set mean image to 0 as mean is now being handled within the model.
X_train_mean_img = np.zeros_like(X_train_mean_img)

all_data = {
    'X_train': X_train,
    'X_val': X_val,
    'y_train': y_train,
    'y_val': y_val,
    'X_test': X_test,
    'X_train_mean_img': X_train_mean_img
}

assert X_train_mean_img == np.array([0.])
assert X_train.shape == (3200, 101, 101, 1)
assert y_train.shape == (3200, 101, 101, 1)
assert depth_train.shape == (3200, 1)

In [0]:
ckp_prefix_list = [
    'salt_model_v37.42_high_lovasz_loss_cv0_2018_10_09_21_41_43.ckp',
    'salt_model_v37.42_high_lovasz_loss_cv1_2018_10_09_22_08_30.ckp',
    'salt_model_v37.42_high_lovasz_loss_cv2_2018_10_10_07_01_06.ckp',
    'salt_model_v37.42_high_lovasz_loss_cv3_2018_10_09_22_26_37.ckp',
    'salt_model_v37.42_high_lovasz_loss_cv4_2018_10_10_07_28_55.ckp'    
]

In [0]:
ckp_prefix_list = [
    'salt_model_v37.42_high_lovasz_loss_cv0_2018_10_09_21_41_43.ckp',
    'salt_model_v37.42_high_lovasz_loss_cv1_2018_10_09_22_08_30.ckp'    
]

In [15]:
cd ../salt_net

/content/salt_net


In [31]:
loaded_model = UResNet(pretrained=False)
model_state_dict = torch.load(join_files(model_prefix_list[0], '.', returnFileObject=True, removeChunks=False))
loaded_model.load_state_dict(model_state_dict['model'])

ResNetnot using pretrained weights.


In [0]:
loaded_model.eval()
assert loaded_model.training == False
if torch.cuda.is_available():
    loaded_model.cuda()

In [0]:
def model_ensembler(loaded_model, ckp_prefix_list, X, y, depth, X_train_mean_img, batch_size):
  y_pred_score_ens = torch.zeros_like(torch.from_numpy(X.squeeze())).float().type(dtype)
  y_pred_mask_ens = torch.zeros_like(torch.from_numpy(X.squeeze())).type(dtype).gt(0)
  
  for ckp in ckp_prefix_list:
    log.info(f'load checkpoint {ckp}...')
    model_state_dict = torch.load(join_files(ckp, '.', returnFileObject=True, removeChunks=False))
    loaded_model.load_state_dict(model_state_dict['model'])
    loaded_model.eval()
    assert loaded_model.training == False
    if torch.cuda.is_available():
        loaded_model.cuda()
    # predict on X
    dataLoader = DataLoader(SaltDataset(X, y, depth, X_train_mean_img, out_size=128), batch_size=batch_size)

    y_pred = []
    with torch.no_grad():
        for X_batch, y_batch, depth_batch, X_id_batch in dataLoader:
            y_pred.append(loaded_model(X_batch))
    y_pred = torch.cat(y_pred)

    # predict on X flip
    p = Pipeline_Salt()
    p.flip_left_right(probability=1)
    dataLoader_flip = DataLoader(SaltDataset(X, y, depth, X_train_mean_img, out_size=128, out_ch=1,
                                             transform=p.torch_transform()), batch_size=batch_size)
    y_pred_flip = []
    with torch.no_grad():
        for X_batch, y_batch, depth_batch, X_id_batch in dataLoader_flip:
            y_pred_flip.append(loaded_model(X_batch))
    y_pred_flip = torch.cat(y_pred_flip)
    y_pred_flip = torch.flip(y_pred_flip,[2])

    # combine y_pred and y_pred_flip
    y_pred_tta = torch.where(y_pred.abs() > y_pred_flip.abs(), y_pred, y_pred_flip)
    
    y_pred_score_ens += (y_pred + y_pred_flip + y_pred_tta)
    y_pred_mask_ens += (y_pred.gt(0) + y_pred_flip.gt(0) + y_pred_tta.gt(0))

  return (y_pred_score_ens, y_pred_mask_ens)

In [18]:
loaded_model = UResNet(pretrained=False)

ResNetnot using pretrained weights.


In [0]:
batch_size = 64

In [42]:
y_score_ens, y_mask_ens = model_ensembler(loaded_model, ckp_prefix_list, X_val, y_val, depth_val, X_train_mean_img, batch_size)

11/10/2018 00:53:23 - salt_model_v37.42_high_lovasz_loss_cv_ens - INFO - load checkpoint salt_model_v37.42_high_lovasz_loss_cv0_2018_10_09_21_41_43.ckp...
/usr/local/lib/python3.6/dist-packages/torch/nn/functional.py:1890: UserWarning: nn.functional.upsample is deprecated. Use nn.functional.interpolate instead.
  warnings.warn("nn.functional.upsample is deprecated. Use nn.functional.interpolate instead.")
11/10/2018 00:53:50 - salt_model_v37.42_high_lovasz_loss_cv_ens - INFO - load checkpoint salt_model_v37.42_high_lovasz_loss_cv1_2018_10_09_22_08_30.ckp...
11/10/2018 00:54:16 - salt_model_v37.42_high_lovasz_loss_cv_ens - INFO - load checkpoint salt_model_v37.42_high_lovasz_loss_cv2_2018_10_10_07_01_06.ckp...
11/10/2018 00:54:42 - salt_model_v37.42_high_lovasz_loss_cv_ens - INFO - load checkpoint salt_model_v37.42_high_lovasz_loss_cv3_2018_10_09_22_26_37.ckp...
11/10/2018 00:55:09 - salt_model_v37.42_high_lovasz_loss_cv_ens - INFO - load checkpoint salt_model_v37.42_high_lovasz_loss_cv

In [43]:
y_mask_ens.max()

tensor(15, device='cuda:0', dtype=torch.uint8)

In [44]:
y_pred_adj = adjust_predictions(0, X_val, y_score_ens.gt(0), y_val.squeeze())

11/10/2018 00:56:51 - salt_model_v37.42_high_lovasz_loss_cv_ens - INFO - IOU score before: 0.8919999999999999, IOU Score after:0.8919999999999999


In [50]:
y_pred_adj = adjust_predictions(0, X_val, y_mask_ens.ge(9), y_val.squeeze())

11/10/2018 00:57:43 - salt_model_v37.42_high_lovasz_loss_cv_ens - INFO - IOU score before: 0.89575, IOU Score after:0.89575


In [28]:
y_pred_adj = adjust_predictions(0, X_val, y_score_ens.gt(0), y_val.squeeze())

11/10/2018 00:49:56 - salt_model_v37.42_high_lovasz_loss_cv_ens - INFO - IOU score before: 0.8830000000000001, IOU Score after:0.8830000000000001


In [39]:
y_pred_adj = adjust_predictions(0, X_val, y_mask_ens.ge(3), y_val.squeeze())

11/10/2018 00:52:47 - salt_model_v37.42_high_lovasz_loss_cv_ens - INFO - IOU score before: 0.88425, IOU Score after:0.88425


In [0]:
dataLoader = DataLoader(SaltDataset(X_val, y_val, depth_val, X_train_mean_img, out_size=128), batch_size=16)

In [30]:
y_pred = []
with torch.no_grad():
    for X_batch, y_batch, depth_batch, X_id_batch in dataLoader:
        y_pred.append(loaded_model(X_batch))
y_pred = torch.cat(y_pred)

/usr/local/lib/python3.6/dist-packages/torch/nn/functional.py:1890: UserWarning: nn.functional.upsample is deprecated. Use nn.functional.interpolate instead.
  warnings.warn("nn.functional.upsample is deprecated. Use nn.functional.interpolate instead.")


In [31]:
p = Pipeline_Salt()
p.flip_left_right(probability=1)
dataLoader_flip = DataLoader(SaltDataset(X_val, y_val, depth_val, X_train_mean_img, out_size=128, out_ch=1,
                           transform=p.torch_transform()), batch_size=16)
y_pred_flip = []

with torch.no_grad():
    for X_batch, y_batch, depth_batch, X_id_batch in dataLoader_flip:
        y_pred_flip.append(loaded_model(X_batch))
y_pred_flip = torch.cat(y_pred_flip)
y_pred_flip = torch.flip(y_pred_flip,[2])

/usr/local/lib/python3.6/dist-packages/torch/nn/functional.py:1890: UserWarning: nn.functional.upsample is deprecated. Use nn.functional.interpolate instead.
  warnings.warn("nn.functional.upsample is deprecated. Use nn.functional.interpolate instead.")


In [0]:
y_pred_tta = torch.where(y_pred.abs() > y_pred_flip.abs(), y_pred, y_pred_flip)

In [33]:
y_pred_adj = adjust_predictions(0, X_val, y_pred.gt(0), y_val.squeeze())

11/10/2018 00:51:00 - salt_model_v37.42_high_lovasz_loss_cv_ens - INFO - IOU score before: 0.8726250000000002, IOU Score after:0.8726250000000002


In [34]:
y_pred_adj = adjust_predictions(0, X_val, y_pred_flip.gt(0), y_val.squeeze())

11/10/2018 00:51:02 - salt_model_v37.42_high_lovasz_loss_cv_ens - INFO - IOU score before: 0.877125, IOU Score after:0.877125


In [36]:
y_pred_adj = adjust_predictions(0, X_val, y_pred_tta.gt(0), y_val.squeeze())

11/10/2018 00:51:53 - salt_model_v37.42_high_lovasz_loss_cv_ens - INFO - IOU score before: 0.8802499999999999, IOU Score after:0.8802499999999999


In [46]:
(y_pred + y_pred_flip + y_pred_tta)

torch.Size([800, 101, 101])

In [0]:
y_pred_score_ens = torch.zeros_like(torch.from_numpy(X_val.squeeze())).float().type(dtype)
y_pred_mask_ens = torch.zeros_like(torch.from_numpy(X_val.squeeze())).type(dtype).gt(0)

In [0]:
y_pred_score_ens += y_pred
y_pred_score_ens += y_pred_flip
y_pred_score_ens += y_pred_tta
y_pred_mask_ens += y_pred.gt(0)
y_pred_mask_ens += y_pred_flip.gt(0)
y_pred_mask_ens += y_pred_tta.gt(0)

In [81]:
y_pred_mask_ens

tensor([[[0, 0, 0,  ..., 0, 0, 0],
         [0, 0, 0,  ..., 0, 0, 0],
         [0, 0, 0,  ..., 0, 0, 0],
         ...,
         [3, 3, 3,  ..., 0, 0, 0],
         [3, 3, 3,  ..., 0, 0, 0],
         [3, 3, 3,  ..., 0, 0, 0]],

        [[0, 0, 0,  ..., 0, 0, 0],
         [0, 0, 0,  ..., 0, 0, 0],
         [0, 0, 0,  ..., 0, 0, 0],
         ...,
         [0, 0, 0,  ..., 0, 0, 0],
         [0, 0, 0,  ..., 0, 0, 0],
         [0, 0, 0,  ..., 0, 0, 0]],

        [[3, 3, 3,  ..., 3, 3, 3],
         [3, 3, 3,  ..., 3, 3, 3],
         [3, 3, 3,  ..., 3, 3, 3],
         ...,
         [0, 0, 0,  ..., 3, 3, 3],
         [0, 0, 0,  ..., 3, 3, 3],
         [0, 0, 0,  ..., 3, 3, 3]],

        ...,

        [[0, 0, 0,  ..., 0, 0, 0],
         [0, 0, 0,  ..., 0, 0, 0],
         [0, 0, 0,  ..., 0, 0, 0],
         ...,
         [0, 0, 0,  ..., 0, 0, 0],
         [0, 0, 0,  ..., 0, 0, 0],
         [0, 0, 0,  ..., 0, 0, 0]],

        [[0, 0, 0,  ..., 0, 0, 0],
         [0, 0, 0,  ..., 0, 0, 0],
         [0,

### Set model to evaluation mode

In [0]:
loaded_model.eval()
assert loaded_model.training == False

In [0]:
val_dataLoader = DataLoader(SaltDataset(X_val, y_val, depth_val, X_train_mean_img, out_size=128), batch_size=16)

In [0]:
if torch.cuda.is_available():
    loaded_model.cuda()

### Eval model on Val data set

In [0]:
y_val_pred = []

with torch.no_grad():
    for X_val_batch, y_val_batch, depth_val_batch, X_val_id_batch in val_dataLoader:
        y_val_pred.append(loaded_model(X_val_batch))
y_val_pred = torch.cat(y_val_pred)

/usr/local/lib/python3.6/dist-packages/torch/nn/functional.py:1890: UserWarning: nn.functional.upsample is deprecated. Use nn.functional.interpolate instead.
  warnings.warn("nn.functional.upsample is deprecated. Use nn.functional.interpolate instead.")


In [0]:
p = Pipeline_Salt()
p.flip_left_right(probability=1)
val_dataLoader = DataLoader(SaltDataset(X_val, y_val, depth_val, X_train_mean_img, out_size=128, out_ch=1,
                           transform=p.torch_transform()), batch_size=16)
y_val_pred_flip = []

with torch.no_grad():
    for X_val_batch, y_val_batch, depth_val_batch, X_val_id_batch in val_dataLoader:
        y_val_pred_flip.append(loaded_model(X_val_batch))
y_val_pred_flip = torch.cat(y_val_pred_flip)
y_val_pred_flip = torch.flip(y_val_pred_flip,[2])

/usr/local/lib/python3.6/dist-packages/torch/nn/functional.py:1890: UserWarning: nn.functional.upsample is deprecated. Use nn.functional.interpolate instead.
  warnings.warn("nn.functional.upsample is deprecated. Use nn.functional.interpolate instead.")


#### merge normal pred and hflip pred

In [0]:
y_val_pred_ens = torch.where(y_val_pred.abs() > y_val_pred_flip.abs(), y_val_pred, y_val_pred_flip)

#### Check normal pred IOU

In [0]:
y_val_pred_adj = adjust_predictions(0, X_val, y_val_pred.gt(0), y_val.squeeze())

10/10/2018 11:21:27 - salt_model_v37.42_high_lovasz_loss_cv0_eval - INFO - IOU score before: 0.82025, IOU Score after:0.82025


#### Check TTA pred IOU

In [0]:
y_val_pred_adj = adjust_predictions(0, X_val, y_val_pred_ens.gt(0), y_val.squeeze())

10/10/2018 11:21:28 - salt_model_v37.42_high_lovasz_loss_cv0_eval - INFO - IOU score before: 0.8186250000000002, IOU Score after:0.8186250000000002


### Find best mask cutoff

In [0]:
results=[]
for cut_off in np.r_[-0.1:0.1:50j]:
  print(cut_off)
  results.append(calc_mean_iou(adjust_predictions(0, X_val, y_val_pred_ens.gt(cut_off), y_val.squeeze()), y_val.squeeze()))

10/10/2018 11:21:30 - salt_model_v37.42_high_lovasz_loss_cv0_eval - INFO - IOU score before: 0.8173750000000001, IOU Score after:0.8173750000000001


-0.1


10/10/2018 11:21:30 - salt_model_v37.42_high_lovasz_loss_cv0_eval - INFO - IOU score before: 0.8173750000000001, IOU Score after:0.8173750000000001


-0.09591836734693879


10/10/2018 11:21:31 - salt_model_v37.42_high_lovasz_loss_cv0_eval - INFO - IOU score before: 0.8173750000000001, IOU Score after:0.8173750000000001


...

0.08775510204081635
0.09183673469387757


10/10/2018 11:21:43 - salt_model_v37.42_high_lovasz_loss_cv0_eval - INFO - IOU score before: 0.8182499999999999, IOU Score after:0.8182499999999999
10/10/2018 11:21:44 - salt_model_v37.42_high_lovasz_loss_cv0_eval - INFO - IOU score before: 0.8182499999999999, IOU Score after:0.8182499999999999


0.09591836734693879


10/10/2018 11:21:44 - salt_model_v37.42_high_lovasz_loss_cv0_eval - INFO - IOU score before: 0.8182499999999999, IOU Score after:0.8182499999999999


0.1


In [0]:
MASK_CUTOFF = np.r_[-0.1:0.1:50j][np.argmax(results)]
ZERO_MASK_CUTOFF = 0

In [0]:
log.info(f'MASK_CUTOFF: {MASK_CUTOFF}')
log.info(f'ZERO_MASK_CUTOFF: {ZERO_MASK_CUTOFF}')

10/10/2018 11:21:47 - salt_model_v37.42_high_lovasz_loss_cv0_eval - INFO - MASK_CUTOFF: -0.026530612244897958
10/10/2018 11:21:47 - salt_model_v37.42_high_lovasz_loss_cv0_eval - INFO - ZERO_MASK_CUTOFF: 0


In [0]:
y_val_pred_adj = adjust_predictions(ZERO_MASK_CUTOFF, X_val, y_val_pred_ens.gt(MASK_CUTOFF), y_val.squeeze())

10/10/2018 11:21:48 - salt_model_v37.42_high_lovasz_loss_cv0_eval - INFO - IOU score before: 0.81875, IOU Score after:0.81875


## Make predictions on test set using TTA

In [0]:
import numpy as np

### Predict using original image

In [0]:
test_dataLoader = DataLoader(SaltDataset(X_test, np.zeros_like(X_test), depth_test, X_train_mean_img, out_size=128), batch_size=32)

In [0]:
y_test_pred_raw = []
with torch.no_grad():
    for X_test_batch, y_test_batch, depth_test_batch, X_test_id_batch in test_dataLoader:
        y_test_pred_raw.append(loaded_model(X_test_batch))
y_test_pred = torch.cat(y_test_pred_raw)

/usr/local/lib/python3.6/dist-packages/torch/nn/functional.py:1890: UserWarning: nn.functional.upsample is deprecated. Use nn.functional.interpolate instead.
  warnings.warn("nn.functional.upsample is deprecated. Use nn.functional.interpolate instead.")


### Predict using flipped images

In [0]:
p = Pipeline_Salt()
p.flip_left_right(probability=1)
test_dataLoader = DataLoader(SaltDataset(X_test, np.zeros_like(X_test), depth_test, X_train_mean_img, out_size=128, out_ch=1,
                           transform=p.torch_transform()), batch_size=32)
y_test_pred_flip = []

with torch.no_grad():
    for X_test_batch, y_test_batch, depth_test_batch, X_test_id_batch in test_dataLoader:
        y_test_pred_flip.append(loaded_model(X_test_batch))
y_test_pred_flip = torch.cat(y_test_pred_flip)
y_test_pred_flip = torch.flip(y_test_pred_flip,[2])

/usr/local/lib/python3.6/dist-packages/torch/nn/functional.py:1890: UserWarning: nn.functional.upsample is deprecated. Use nn.functional.interpolate instead.
  warnings.warn("nn.functional.upsample is deprecated. Use nn.functional.interpolate instead.")


### Combine non-flip and flip predictions

In [0]:
y_test_pred_ens = torch.where(y_test_pred.abs() > y_test_pred_flip.abs(), y_test_pred, y_test_pred_flip)

### Adjust predictions

In [0]:
#MASK_CUTOFF = 0
#ZERO_MASK_CUTOFF = 0

In [0]:
log.info(f'MASK_CUTOFF:{MASK_CUTOFF}, ZERO_MASK_CUTOFF:{ZERO_MASK_CUTOFF}')

10/10/2018 11:40:11 - salt_model_v37.42_high_lovasz_loss_cv0_eval - INFO - MASK_CUTOFF:-0.026530612244897958, ZERO_MASK_CUTOFF:0


In [0]:
y_test_pred_adj = adjust_predictions(ZERO_MASK_CUTOFF, X_test, y_test_pred_ens.gt(MASK_CUTOFF))


# Predict on Train Data

### Train Val data split

In [0]:
from sklearn.model_selection import StratifiedKFold

In [0]:
fold = 0

In [0]:
skf = StratifiedKFold(n_splits=5, shuffle=True, random_state=0)
df_train_mask = pd.DataFrame((np_train_all_mask/255).sum((1,2,3)), columns=['mask_pix'])
df_train_mask.mask_pix = df_train_mask.mask_pix.round(-2)
nfolds = list(skf.split(df_train_mask.index.tolist(), df_train_mask.mask_pix))

In [0]:
X_train_ids = np.r_[nfolds[fold]][:3200]
X_val_ids = np.r_[nfolds[fold]][3200:]

In [26]:
log.info(f'np.sum(X_train_ids): {np.sum(X_train_ids)}')
log.info(f'np.sum(X_val_ids): {np.sum(X_val_ids)}')

10/10/2018 23:15:18 - salt_model_v37.42_high_lovasz_loss_cv_ens - INFO - np.sum(X_train_ids): 6319371
10/10/2018 23:15:18 - salt_model_v37.42_high_lovasz_loss_cv_ens - INFO - np.sum(X_val_ids): 1678629


In [0]:
#df_train_mask = pd.DataFrame((np_train_all_mask/255).sum((1,2,3)), columns=['mask_pix'])
#df_train_mask.mask_pix = df_train_mask.mask_pix.round(-2)

#X_train_ids, X_val_ids = (
#    train_test_split(df_train_mask.index.tolist(), 
#                     test_size=0.20,
#                     stratify = df_train_mask.mask_pix,
#                     random_state=0)
#)

X_train = np_train_all[X_train_ids]
X_val = np_train_all[X_val_ids]
y_train = np_train_all_mask[X_train_ids]
y_val = np_train_all_mask[X_val_ids]
depth_train = (
    misc_data['df_train_all_depth']
    .reindex(np.array(misc_data['np_train_all_ids'])[X_train_ids])
)
depth_val = (
    misc_data['df_train_all_depth']
    .reindex(np.array(misc_data['np_train_all_ids'])[X_val_ids])
)
depth_test = (
    misc_data['df_train_all_depth']
    .reindex(np.array(misc_data['np_test_ids']))
)
#X_train_mean_img = X_train.mean(0).astype(np.float32)
#X_train_mean_img = X_train.mean((0,1,2)).astype(np.float32)
X_train_mean_img = np.clip(np_train_all/255, 0, 1).mean((0,1,2)).astype(np.float32)
#set mean image to 0 as mean is now being handled within the model.
X_train_mean_img = np.zeros_like(X_train_mean_img)

all_data = {
    'X_train': X_train,
    'X_val': X_val,
    'y_train': y_train,
    'y_val': y_val,
    'X_test': X_test,
    'X_train_mean_img': X_train_mean_img
}

assert X_train_mean_img == np.array([0.])
assert X_train.shape == (3200, 101, 101, 1)
assert y_train.shape == (3200, 101, 101, 1)
assert depth_train.shape == (3200, 1)

## Train the full with full dataset

In [0]:
log.info(f'{run_name}, based on salt_model_v37.4_high_lovasz_loss. 5 folds CV.')

09/10/2018 11:09:45 - salt_model_v37.42_high_lovasz_loss_cv1 - INFO - salt_model_v37.42_high_lovasz_loss_cv1, based on salt_model_v37.4_high_lovasz_loss. 5 folds CV.


In [0]:
aug_pipeline_config = '''
p = Pipeline_Salt()
p.flip_left_right(probability=0.5)
p.rotate90(0.25)
p.rotate270(0.25)
p.random_brightness(probability=0.5, min_factor=0.8, max_factor=1.2)
p.random_contrast(probability=0.5, min_factor=0.8, max_factor=1.2)
p.rotate_random_align(probability=0.5)
p.crop_random_align(probability=0.5, min_factor=0.6, max_factor=1.0, mask_diff_pct=0.2)
'''

In [0]:
data_loader_config = '''
train_data_params = {'batch_size': 32,
                     #'sampler': weighted_sampler,
                     'shuffle': True,
                     'drop_last': False}

val_data_params = {'batch_size': 32,
                   'shuffle': True,
                   'drop_last': False}

train_dataLoader = (
    DataLoader(SaltDataset(X_train, y_train, depth_train,
                           np.zeros_like(X_train_mean_img), out_size=128,  out_ch=1,
                           transform=p.torch_transform()), **train_data_params)
)

val_dataLoader = (
    DataLoader(SaltDataset(X_val, y_val, depth_val, 
                           np.zeros_like(X_train_mean_img), out_size=128, out_ch=1), **val_data_params)
)

dataloaders = {'train': train_dataLoader, 'val':val_dataLoader}
'''

In [0]:
model_config = '''
saltnet = UResNet(pretrained=True)
loss_fn_bce = nn.BCEWithLogitsLoss(pos_weight=torch.tensor(2.0).type(dtype))
#loss_focal = FocalLoss(0.25, 2, logits=True)
loss_lovasz_hinge = LovaszHingeLoss()
resnet_params = (
    list(saltnet.conv1.parameters()) + 
    list(saltnet.encoder2.parameters()) + 
    list(saltnet.encoder3.parameters()) + 
    list(saltnet.encoder4.parameters()) + 
    list(saltnet.encoder5.parameters())
)

unet_params = (
    list(saltnet.center.parameters()) + 
    list(saltnet.decoder5.parameters()) + 
    list(saltnet.decoder4.parameters()) + 
    list(saltnet.decoder3.parameters()) + 
    list(saltnet.decoder2.parameters()) + 
    list(saltnet.decoder1.parameters())  + 
    list(saltnet.se_f.parameters()) + 
    list(saltnet.outc.parameters())
)
optimizer = optim.Adam([    
    {'params': resnet_params, 'lr': 1e-4},
    {'params': unet_params, 'lr': 1e-3},
], weight_decay=0.0001)


scheduler = PolyLR(optimizer, [1e-4, 1e-3], lr_decay_iter=1, max_iter=150, power=0.9)
model_save_name = model_file_suffix
log.info(model_save_name)
'''

In [0]:
train_params_config = '''
train_params = {
    'model_save_name': model_save_name,
    'save_model_every': 20,
    'save_log_every': 2,
    'num_epochs': 150,
    'log': log,
    'mask_cutoff': 0.,
    'model_save_iou_threshold': 0.81
    }
'''

In [0]:
model_run_config = '''
train_model(saltnet, dataloaders, (loss_fn_bce, loss_lovasz_hinge), (1, 0.3), optimizer, scheduler, train_params, all_data)
'''

In [0]:
config_list = [aug_pipeline_config, data_loader_config, model_config, train_params_config, model_run_config]

In [0]:
setup_train(config_list)

## Load Trained Model

In [0]:
ls

In [0]:
cd ../salt_net

/content/salt_net


In [0]:
loaded_model = UResNet(pretrained=False)

model_file_suffix = "salt_model_v37.42_high_lovasz_loss_cv0_2018_10_09_21_41_43.ckp"
model_state_dict = torch.load(join_files(model_file_suffix, '.', returnFileObject=True, removeChunks=False))
loaded_model.load_state_dict(model_state_dict['model'])

ResNetnot using pretrained weights.


In [0]:
loaded_model = UResNet(pretrained=False)

model_file_suffix = "salt_model_v37.42_high_lovasz_loss_cv0_2018_10_09_21_41_43.ckp"
model_state_dict = torch.load(join_files(model_file_suffix, '.', returnFileObject=True, removeChunks=False))
loaded_model.load_state_dict(model_state_dict['model'])

### Make Predictions on validation set

### Set model to evaluation mode

In [0]:
loaded_model.eval()
assert loaded_model.training == False

In [0]:
val_dataLoader = DataLoader(SaltDataset(X_val, y_val, depth_val, X_train_mean_img, out_size=128), batch_size=16)

In [0]:
if torch.cuda.is_available():
    loaded_model.cuda()

### Eval model on Val data set

In [0]:
y_val_pred = []

with torch.no_grad():
    for X_val_batch, y_val_batch, depth_val_batch, X_val_id_batch in val_dataLoader:
        y_val_pred.append(loaded_model(X_val_batch))
y_val_pred = torch.cat(y_val_pred)

/usr/local/lib/python3.6/dist-packages/torch/nn/functional.py:1890: UserWarning: nn.functional.upsample is deprecated. Use nn.functional.interpolate instead.
  warnings.warn("nn.functional.upsample is deprecated. Use nn.functional.interpolate instead.")


In [0]:
p = Pipeline_Salt()
p.flip_left_right(probability=1)
val_dataLoader = DataLoader(SaltDataset(X_val, y_val, depth_val, X_train_mean_img, out_size=128, out_ch=1,
                           transform=p.torch_transform()), batch_size=16)
y_val_pred_flip = []

with torch.no_grad():
    for X_val_batch, y_val_batch, depth_val_batch, X_val_id_batch in val_dataLoader:
        y_val_pred_flip.append(loaded_model(X_val_batch))
y_val_pred_flip = torch.cat(y_val_pred_flip)
y_val_pred_flip = torch.flip(y_val_pred_flip,[2])

/usr/local/lib/python3.6/dist-packages/torch/nn/functional.py:1890: UserWarning: nn.functional.upsample is deprecated. Use nn.functional.interpolate instead.
  warnings.warn("nn.functional.upsample is deprecated. Use nn.functional.interpolate instead.")


#### merge normal pred and hflip pred

In [0]:
y_val_pred_ens = torch.where(y_val_pred.abs() > y_val_pred_flip.abs(), y_val_pred, y_val_pred_flip)

#### Check normal pred IOU

In [0]:
y_val_pred_adj = adjust_predictions(0, X_val, y_val_pred.gt(0), y_val.squeeze())

10/10/2018 11:21:27 - salt_model_v37.42_high_lovasz_loss_cv0_eval - INFO - IOU score before: 0.82025, IOU Score after:0.82025


#### Check TTA pred IOU

In [0]:
y_val_pred_adj = adjust_predictions(0, X_val, y_val_pred_ens.gt(0), y_val.squeeze())

10/10/2018 11:21:28 - salt_model_v37.42_high_lovasz_loss_cv0_eval - INFO - IOU score before: 0.8186250000000002, IOU Score after:0.8186250000000002


### Find best mask cutoff

In [0]:
results=[]
for cut_off in np.r_[-0.1:0.1:50j]:
  print(cut_off)
  results.append(calc_mean_iou(adjust_predictions(0, X_val, y_val_pred_ens.gt(cut_off), y_val.squeeze()), y_val.squeeze()))

10/10/2018 11:21:30 - salt_model_v37.42_high_lovasz_loss_cv0_eval - INFO - IOU score before: 0.8173750000000001, IOU Score after:0.8173750000000001


-0.1


10/10/2018 11:21:30 - salt_model_v37.42_high_lovasz_loss_cv0_eval - INFO - IOU score before: 0.8173750000000001, IOU Score after:0.8173750000000001


-0.09591836734693879


10/10/2018 11:21:31 - salt_model_v37.42_high_lovasz_loss_cv0_eval - INFO - IOU score before: 0.8173750000000001, IOU Score after:0.8173750000000001


...

0.08775510204081635
0.09183673469387757


10/10/2018 11:21:43 - salt_model_v37.42_high_lovasz_loss_cv0_eval - INFO - IOU score before: 0.8182499999999999, IOU Score after:0.8182499999999999
10/10/2018 11:21:44 - salt_model_v37.42_high_lovasz_loss_cv0_eval - INFO - IOU score before: 0.8182499999999999, IOU Score after:0.8182499999999999


0.09591836734693879


10/10/2018 11:21:44 - salt_model_v37.42_high_lovasz_loss_cv0_eval - INFO - IOU score before: 0.8182499999999999, IOU Score after:0.8182499999999999


0.1


In [0]:
MASK_CUTOFF = np.r_[-0.1:0.1:50j][np.argmax(results)]
ZERO_MASK_CUTOFF = 0

In [0]:
log.info(f'MASK_CUTOFF: {MASK_CUTOFF}')
log.info(f'ZERO_MASK_CUTOFF: {ZERO_MASK_CUTOFF}')

10/10/2018 11:21:47 - salt_model_v37.42_high_lovasz_loss_cv0_eval - INFO - MASK_CUTOFF: -0.026530612244897958
10/10/2018 11:21:47 - salt_model_v37.42_high_lovasz_loss_cv0_eval - INFO - ZERO_MASK_CUTOFF: 0


In [0]:
y_val_pred_adj = adjust_predictions(ZERO_MASK_CUTOFF, X_val, y_val_pred_ens.gt(MASK_CUTOFF), y_val.squeeze())

10/10/2018 11:21:48 - salt_model_v37.42_high_lovasz_loss_cv0_eval - INFO - IOU score before: 0.81875, IOU Score after:0.81875


## Make predictions on test set using TTA

In [0]:
import numpy as np

### Predict using original image

In [0]:
test_dataLoader = DataLoader(SaltDataset(X_test, np.zeros_like(X_test), depth_test, X_train_mean_img, out_size=128), batch_size=32)

In [0]:
y_test_pred_raw = []
with torch.no_grad():
    for X_test_batch, y_test_batch, depth_test_batch, X_test_id_batch in test_dataLoader:
        y_test_pred_raw.append(loaded_model(X_test_batch))
y_test_pred = torch.cat(y_test_pred_raw)

/usr/local/lib/python3.6/dist-packages/torch/nn/functional.py:1890: UserWarning: nn.functional.upsample is deprecated. Use nn.functional.interpolate instead.
  warnings.warn("nn.functional.upsample is deprecated. Use nn.functional.interpolate instead.")


### Predict using flipped images

In [0]:
p = Pipeline_Salt()
p.flip_left_right(probability=1)
test_dataLoader = DataLoader(SaltDataset(X_test, np.zeros_like(X_test), depth_test, X_train_mean_img, out_size=128, out_ch=1,
                           transform=p.torch_transform()), batch_size=32)
y_test_pred_flip = []

with torch.no_grad():
    for X_test_batch, y_test_batch, depth_test_batch, X_test_id_batch in test_dataLoader:
        y_test_pred_flip.append(loaded_model(X_test_batch))
y_test_pred_flip = torch.cat(y_test_pred_flip)
y_test_pred_flip = torch.flip(y_test_pred_flip,[2])

/usr/local/lib/python3.6/dist-packages/torch/nn/functional.py:1890: UserWarning: nn.functional.upsample is deprecated. Use nn.functional.interpolate instead.
  warnings.warn("nn.functional.upsample is deprecated. Use nn.functional.interpolate instead.")


### Combine non-flip and flip predictions

In [0]:
y_test_pred_ens = torch.where(y_test_pred.abs() > y_test_pred_flip.abs(), y_test_pred, y_test_pred_flip)

### Adjust predictions

In [0]:
#MASK_CUTOFF = 0
#ZERO_MASK_CUTOFF = 0

In [0]:
log.info(f'MASK_CUTOFF:{MASK_CUTOFF}, ZERO_MASK_CUTOFF:{ZERO_MASK_CUTOFF}')

10/10/2018 11:40:11 - salt_model_v37.42_high_lovasz_loss_cv0_eval - INFO - MASK_CUTOFF:-0.026530612244897958, ZERO_MASK_CUTOFF:0


In [0]:
y_test_pred_adj = adjust_predictions(ZERO_MASK_CUTOFF, X_test, y_test_pred_ens.gt(MASK_CUTOFF))


# Predict on Train Data

In [0]:
train_dataLoader = DataLoader(SaltDataset(X_train, y_train, depth_train, X_train_mean_img, out_size=128), batch_size=32)
y_train_pred = []

with torch.no_grad():
    for X_train_batch, y_train_batch, depth_train_batch, X_train_id_batch in train_dataLoader:
        y_train_pred.append(loaded_model(X_train_batch))
y_train_pred = torch.cat(y_train_pred)

/usr/local/lib/python3.6/dist-packages/torch/nn/functional.py:1890: UserWarning: nn.functional.upsample is deprecated. Use nn.functional.interpolate instead.
  warnings.warn("nn.functional.upsample is deprecated. Use nn.functional.interpolate instead.")


In [0]:
p = Pipeline_Salt()
p.flip_left_right(probability=1)
train_dataLoader = DataLoader(SaltDataset(X_train, y_train, depth_train, X_train_mean_img, out_size=128, out_ch=1,
                           transform=p.torch_transform()), batch_size=32)
y_train_pred_flip = []

with torch.no_grad():
    for X_train_batch, y_train_batch, depth_train_batch, X_train_id_batch in train_dataLoader:
        y_train_pred_flip.append(loaded_model(X_train_batch))
y_train_pred_flip = torch.cat(y_train_pred_flip)
y_train_pred_flip = torch.flip(y_train_pred_flip,[2])

/usr/local/lib/python3.6/dist-packages/torch/nn/functional.py:1890: UserWarning: nn.functional.upsample is deprecated. Use nn.functional.interpolate instead.
  warnings.warn("nn.functional.upsample is deprecated. Use nn.functional.interpolate instead.")


In [0]:
y_train_pred_ens = torch.where(y_train_pred.abs() > y_train_pred_flip.abs(), y_train_pred, y_train_pred_flip)

### Prepare for submission

In [0]:
y_test_pred_rle = rle_encoder3d(y_test_pred_adj)

In [0]:
df_test_rle = pd.DataFrame(index=misc_data['np_test_ids'], data=y_test_pred_rle).reset_index()

In [0]:
df_test_rle.columns = ['id', 'rle_mask']

In [0]:
sub_fname = f'submission_{get_current_time_as_fname()}.csv'

In [0]:
df_test_rle.to_csv(sub_fname, index=False)

In [0]:
log.info(f'submission file {sub_fname} generated.')

09/10/2018 20:57:10 - salt_model_v37.42_high_lovasz_loss_cv1 - INFO - submission file submission_2018_10_10_07_57_06.csv generated.


In [0]:
from google.colab import files

In [0]:
files.download(sub_fname) 

In [0]:
log.info(f'submission file {sub_fname} downloaded.')

09/10/2018 20:57:32 - salt_model_v37.42_high_lovasz_loss_cv1 - INFO - submission file submission_2018_10_10_07_57_06.csv downloaded.


In [0]:
data_dict = {'run_name': run_name,
             'y_val_pred': y_val_pred,
             'y_val_pred_flip': y_val_pred_flip,
             'y_train_pred': y_train_pred,
             'y_train_pred_flip': y_train_pred_flip,
             'y_test_pred': y_test_pred,
             'y_test_pred_flip': y_test_pred_flip}

In [0]:
import pickle

with open(f'{run_name}_{get_current_time_as_fname()}.pickle', 'wb') as handle:
    pickle.dump(data_dict, handle, protocol=pickle.HIGHEST_PROTOCOL)



In [0]:
push_model_to_git()

10/10/2018 11:57:42 - salt_model_v37.42_high_lovasz_loss_cv0_eval - INFO - Pushing model state to git.


/content/salt_net
Already up-to-date.
[master d3860ea] save model state.
 3 files changed, 59 insertions(+)
 create mode 100644 logs/SaltNet_2018_10_10_22h.log
 create mode 100644 logs/salt_model_v37.42_high_lovasz_loss_cv0_eval_2018_10_10_22h.log
 create mode 100644 salt_model_v37.42_high_lovasz_loss_cv0_eval_2018_10_10_22_53_02.pickle
Counting objects: 5, done.
Delta compression using up to 2 threads.
Compressing objects: 100% (4/4), done.
Writing objects: 100% (5/5), 1.47 GiB | 45.25 MiB/s, done.
Total 5 (delta 2), reused 1 (delta 0)
remote: Resolving deltas: 100% (2/2), completed with 2 local objects.
remote: error: GH001: Large files detected. You may want to try Git Large File Storage - https://git-lfs.github.com.
remote: error: Trace: b095bf729957376f11e10c2edb34208b
remote: error: See http://git.io/iEPt8g for more information.
remote: error: File salt_model_v37.42_high_lovasz_loss_cv0_eval_2018_10_10_22_53_02.pickle is 1712.21 MB; this exceeds GitHub's file size limit of 100.00

In [0]:
ls

logs/
salt_model_v37.42_high_lovasz_loss_cv0_2018_10_09_21_41_43.ckp-chunk-10-Of-10
salt_model_v37.42_high_lovasz_loss_cv0_2018_10_09_21_41_43.ckp-chunk-1-Of-10
salt_model_v37.42_high_lovasz_loss_cv0_2018_10_09_21_41_43.ckp-chunk-2-Of-10
salt_model_v37.42_high_lovasz_loss_cv0_2018_10_09_21_41_43.ckp-chunk-3-Of-10
salt_model_v37.42_high_lovasz_loss_cv0_2018_10_09_21_41_43.ckp-chunk-4-Of-10
salt_model_v37.42_high_lovasz_loss_cv0_2018_10_09_21_41_43.ckp-chunk-5-Of-10
salt_model_v37.42_high_lovasz_loss_cv0_2018_10_09_21_41_43.ckp-chunk-6-Of-10
salt_model_v37.42_high_lovasz_loss_cv0_2018_10_09_21_41_43.ckp-chunk-7-Of-10
salt_model_v37.42_high_lovasz_loss_cv0_2018_10_09_21_41_43.ckp-chunk-8-Of-10
salt_model_v37.42_high_lovasz_loss_cv0_2018_10_09_21_41_43.ckp-chunk-9-Of-10
salt_model_v37.42_high_lovasz_loss_cv0_eval_2018_10_10_22_53_02.pickle
salt_model_v37.42_high_lovasz_loss_cv1_2018_10_09_22_08_30.ckp-chunk-10-Of-10
salt_model_v37.42_high_lovasz_loss_cv1_2018_10_09_22_08_30.ckp-chunk-1-Of-